In [ ]:
import os
import re
import json
import asyncio
import logging
from typing import Optional, List, Dict, Any

import httpx
from pydantic import BaseModel, Field
from openai import AsyncAzureOpenAI
from agents import Agent, Runner, OpenAIChatCompletionsModel, function_tool

# -------------------------
# 0) ENV / CONFIG
# -------------------------
# Set these here OR export them in your shell.
os.environ.setdefault("AZURE_ENDPOINT",   "https://kai0721300033.cognitiveservices.azure.com")
# os.environ.setdefault("AZURE_KEY",        "REPLACE_ME")                 # <-- set real key (or export it)
os.environ.setdefault("API_VERSION",      "2025-01-01-preview")           # try GA (e.g. 2024-10-21) if preview is fussy
os.environ.setdefault("AZURE_DEPLOYMENT", "gpt-4o")                       # <-- your Azure *deployment name*

# Kahua
# os.environ.setdefault("KAHUA_BASIC_AUTH", "Basic REPLACE_ME_BASE64")    # <-- set real Basic header (or export it)
# Include {project_id} placeholder so we can reuse base for root or specific projects:
os.environ.setdefault(
    "KAHUA_QUERY_BASE",
    "https://devdailyservice.kahua.com/v2/domains/AWrightCo/projects/{project_id}/query?returnDefaultAttributes=true"
)
os.environ.setdefault(
    "KAHUA_PROJECT_URL",
    "https://devdailyservice.kahua.com/v2/domains/AWrightCo/projects/0/apps/kahua_AEC_RFI/activities/run"
)

AZURE_ENDPOINT    = os.environ.get("AZURE_ENDPOINT", "")
AZURE_KEY         = os.environ.get("AZURE_KEY")  # may be None if not set
API_VERSION       = os.environ.get("API_VERSION", "")
AZURE_DEPLOYMENT  = os.environ.get("AZURE_DEPLOYMENT", "")

KAHUA_BASIC_AUTH  = os.environ.get("KAHUA_BASIC_AUTH")  # may be None if not set
KAHUA_QUERY_BASE  = os.environ.get("KAHUA_QUERY_BASE", "")
KAHUA_PROJECT_URL = os.environ.get("KAHUA_PROJECT_URL", "")

logging.basicConfig(level=logging.INFO, format="%(levelname)s %(name)s: %(message)s")
log = logging.getLogger("demo")
# Dedicated logger for agent handoffs
handoff_log = logging.getLogger("agents.handoff")
handoff_log.setLevel(logging.INFO)

# def require_env():
#     missing = []
#     if not AZURE_ENDPOINT: missing.append("AZURE_ENDPOINT")
#     if not AZURE_KEY: missing.append("AZURE_KEY")
#     if not API_VERSION: missing.append("API_VERSION")
#     if not AZURE_DEPLOYMENT: missing.append("AZURE_DEPLOYMENT")
#     if not KAHUA_BASIC_AUTH or not KAHUA_BASIC_AUTH.startswith("Basic "): missing.append("KAHUA_BASIC_AUTH('Basic ...')")
#     if not KAHUA_QUERY_BASE: missing.append("KAHUA_QUERY_BASE")
#     if not KAHUA_PROJECT_URL: missing.append("KAHUA_PROJECT_URL")
#     if missing:
#         raise RuntimeError(f"Missing/invalid env: {', '.join(missing)}")
# require_env()

# -------------------------
# 1) AZURE OPENAI CLIENT
# -------------------------
azure_client = AsyncAzureOpenAI(
    api_key=AZURE_KEY,
    azure_endpoint=AZURE_ENDPOINT,
    api_version=API_VERSION,
)

# -------------------------
# 2) SCHEMAS
# -------------------------
class ProjectItem(BaseModel):
    """Schema for project creation"""
    id: Optional[int] = Field(None, description="Kahua Project Id (optional for create)")
    name: str = Field(..., description="Project name (Kahua: Name)")
    description: Optional[str] = Field(None, description="Project description")
    auto_send: bool = Field(True, description="If true, tool can be called immediately by the agent")

# -------------------------
# 3) TOOLS
# -------------------------

@function_tool
async def list_projects(name_contains: Optional[str] = None, project_scope_id: str = "0") -> Dict[str, Any]:
    """
    Return a list of projects. Optionally filter by substring in Name (case-insensitive).
    """
    url = KAHUA_QUERY_BASE.format(project_id=project_scope_id)
    payload = {"PropertyName": "Query", "EntityDef": "kahua_Project.Project"}
    headers = {"Content-Type": "application/json", "Authorization": KAHUA_BASIC_AUTH}

    # Log request
    redacted = f"{KAHUA_BASIC_AUTH[:10]}...redacted"
    log.info("========== KAHUA REQUEST (list_projects) ==========")
    log.info(f"URL: {url}")
    log.info(f"Headers:\n{json.dumps({**headers, 'Authorization': redacted}, indent=2)}")
    log.info(f"Payload:\n{json.dumps(payload, indent=2)}")
    log.info("===================================================")

    async with httpx.AsyncClient(timeout=20.0) as client:
        resp = await client.post(url, headers=headers, json=payload)

    # Log response
    log.info("========== KAHUA RESPONSE (list_projects) =========")
    log.info(f"Status: {resp.status_code}")
    log.info(f"Headers:\n{json.dumps(dict(resp.headers), indent=2)}")
    log.info(f"Body:\n{resp.text[:2000]}")
    log.info("===================================================")

    ctype = (resp.headers.get("content-type") or "").lower()
    data = resp.json() if "application/json" in ctype else {"text": resp.text}

    if resp.status_code >= 400:
        return {"status": "error", "upstream_status": resp.status_code, "body": data}

    items = data.get("items") or data.get("result") or []
    projects = []
    if isinstance(items, list):
        for it in items:
            projects.append({
                "Id": it.get("Id") or it.get("id"),
                "Name": it.get("Name"),
                "Description": it.get("Description"),
                "Status": it.get("Status"),
                "ProjectNumber": it.get("ProjectNumber"),
            })

    if name_contains:
        pat = re.compile(re.escape(name_contains), re.IGNORECASE)
        projects = [p for p in projects if p.get("Name") and pat.search(p["Name"])]

    return {"status": "ok", "count": len(projects), "projects": projects}


@function_tool
async def list_punch_items(project_id: str = "0",
                           location_contains: Optional[str] = None,
                           defect_contains: Optional[str] = None,
                           subject_contains: Optional[str] = None) -> Dict[str, Any]:
    """
    Return a list of PunchList items within a project scope.
    Optional filters are client-side contains matches on common fields.
    """
    url = KAHUA_QUERY_BASE.format(project_id=project_id)
    payload = {"PropertyName": "Query", "EntityDef": "kahua_AEC_PunchList.PunchListItem"}
    headers = {"Content-Type": "application/json", "Authorization": KAHUA_BASIC_AUTH}

    # Log request
    redacted = f"{KAHUA_BASIC_AUTH[:10]}...redacted"
    log.info("========== KAHUA REQUEST (list_punch_items) =======")
    log.info(f"URL: {url}")
    log.info(f"Headers:\n{json.dumps({**headers, 'Authorization': redacted}, indent=2)}")
    log.info(f"Payload:\n{json.dumps(payload, indent=2)}")
    log.info("===================================================")

    async with httpx.AsyncClient(timeout=20.0) as client:
        resp = await client.post(url, headers=headers, json=payload)

    # Log response
    log.info("========== KAHUA RESPONSE (list_punch_items) ======")
    log.info(f"Status: {resp.status_code}")
    log.info(f"Headers:\n{json.dumps(dict(resp.headers), indent=2)}")
    log.info(f"Body:\n{resp.text[:2000]}")
    log.info("===================================================")

    ctype = (resp.headers.get("content-type") or "").lower()
    data = resp.json() if "application/json" in ctype else {"text": resp.text}

    if resp.status_code >= 400:
        return {"status": "error", "upstream_status": resp.status_code, "body": data}

    items = data.get("items") or data.get("result") or []
    rows = []
    if isinstance(items, list):
        for it in items:
            rows.append({
                "Id": it.get("Id") or it.get("id"),
                "Subject": it.get("Subject"),
                "Description": it.get("Description"),
                "Location": it.get("Location") or it.get("Area"),
                "Defect": it.get("Defect"),
                "Status": it.get("Status"),
                "Assignee": it.get("Assignee") or it.get("AssignedTo"),
                "CreatedDate": it.get("CreatedDate") or it.get("CreatedOn") or it.get("CreatedDateTime"),
            })

    def contains(val: Optional[str], needle: Optional[str]) -> bool:
        if not needle:
            return True
        return bool(val) and (needle.lower() in val.lower())

    rows = [
        x for x in rows
        if contains(x.get("Location"), location_contains)
        and contains(x.get("Defect"), defect_contains)
        and contains(x.get("Subject"), subject_contains)
    ]

    return {"status": "ok", "count": len(rows), "punch_items": rows}


@function_tool
async def create_project_in_kahua(project: ProjectItem) -> Dict[str, Any]:
    """
    Create a Project in Kahua (entityDef: kahua_Project.Project) via activities/run.
    """
    entity = {
        "id": 0,
        "hubPath": "kahua_Project.NoWorkflow\\Start",
        "entityDef": "kahua_Project.Project",
        "Name": project.name,
    }
    if project.description:
        entity["Description"] = project.description
    if project.id is not None:
        entity["Id"] = project.id

    payload = {
        "activity": {
            "PropertyName": "Activity",
            "Name": "ContractItem",
            "Flow": [
                {"PropertyName": "Iterate", "Set": "ContractItem", "New": {}, "Existing": {}}
            ],
        },
        "sets": [{"name": "ContractItem", "entities": [entity]}],
    }

    headers = {"Content-Type": "application/json", "Authorization": KAHUA_BASIC_AUTH}

    # Debug: full request log
    redacted_auth = f"{headers['Authorization'][:10]}...redacted"
    log.info("========== KAHUA REQUEST (create_project) =========")
    log.info(f"URL: {KAHUA_PROJECT_URL}")
    log.info(f"Headers:\n{json.dumps({**headers, 'Authorization': redacted_auth}, indent=2)}")
    log.info(f"Payload:\n{json.dumps(payload, indent=2)}")
    log.info("===================================================")

    async with httpx.AsyncClient(timeout=20.0) as client:
        resp = await client.post(KAHUA_PROJECT_URL, headers=headers, json=payload)

    # Debug: full response log
    log.info("========== KAHUA RESPONSE (create_project) ========")
    log.info(f"Status: {resp.status_code}")
    log.info(f"Headers:\n{json.dumps(dict(resp.headers), indent=2)}")
    log.info(f"Body:\n{resp.text[:2000]}")
    log.info("===================================================")

    ctype = (resp.headers.get("content-type") or "").lower()
    body = resp.json() if "application/json" in ctype else {"text": resp.text}

    if resp.status_code >= 400:
        return {
            "status": "error",
            "upstream_status": resp.status_code,
            "upstream_content_type": ctype,
            "upstream_body": body,
            "payload_preview": {"Name": entity.get("Name"), "Description": (entity.get("Description") or "")[:200]},
        }

    return {
        "status": "ok",
        "upstream_status": resp.status_code,
        "kahua_response": body,
        "payload_preview": {"Name": entity.get("Name"), "Description": (entity.get("Description") or "")[:200]},
    }

# -------------------------
# 4) AGENTS
# -------------------------

# Entity Query agent: lists/browses projects/punch items
entity_query_agent = Agent(
    name="Entity Query",
    handoff_description="Search/browse projects or punch items in Kahua",
    instructions=(
        "When the user asks to list/show/find/browse projects, call list_projects "
        "(optionally with name_contains). "
        "When they ask to list punch items, call list_punch_items (optionally with project_id and filters). "
        "Summarize results concisely."
    ),
    model=OpenAIChatCompletionsModel(
        model=AZURE_DEPLOYMENT,
        openai_client=azure_client
    ),
    tools=[list_projects, list_punch_items]
)

# Project Creator agent: returns JSON ONLY and then calls the create tool
class _ProjectItem(ProjectItem):  # just to be explicit in instructions
    pass

project_creation_agent = Agent(
    name="Project Creator",
    handoff_description="Creates projects in Kahua",
    instructions=(
        "Create a new Kahua project. "
        "Return a SINGLE valid JSON object matching ProjectItem (no prose, no markdown). "
        "If user asks to create now or auto_send is true, call create_project_in_kahua with that JSON."
    ),
    model=OpenAIChatCompletionsModel(
        model=AZURE_DEPLOYMENT,
        openai_client=azure_client,
        # If supported by your Agents SDK version, uncomment to enforce JSON:
        # response_format={\"type\": \"json_object\"}
    ),
    output_type=_ProjectItem,
    tools=[create_project_in_kahua],
)

# Triage router
triage_agent = Agent(
    name="Router",
    instructions=(
        "Routing rules:\n"
        "- If the user wants to list/show/find/browse projects or punch items, hand off to Entity Query.\n"
        "- If the user wants to create a project, hand off to Project Creator.\n"
        "- Otherwise answer directly."
    ),
    model=OpenAIChatCompletionsModel(
        model=AZURE_DEPLOYMENT,
        openai_client=azure_client
    ),
    handoffs=[entity_query_agent, project_creation_agent],
)

# -------------------------
# 5) DEMO RUNS
# -------------------------
async def main():
    print("\n--- DEMO A: List projects ---")
    res1 = await Runner.run(triage_agent, "List all projects in Kahua.")
    print(res1.final_output)

    print("\n--- DEMO B: Create project ---")
    res2 = await Runner.run(triage_agent, "Create a project called East Tower Renovation. Description 'Demo via tool'. Send it now.")
    print(res2.final_output)

if __name__ == "__main__":
    try:
        loop = asyncio.get_event_loop()
        if loop.is_running():
            import nest_asyncio
            nest_asyncio.apply()
            asyncio.ensure_future(main())
        else:
            loop.run_until_complete(main())
    except RuntimeError:
        asyncio.run(main())



--- DEMO A: List projects ---


INFO httpx: HTTP Request: POST https://kai0721300033.cognitiveservices.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"
INFO httpx: HTTP Request: POST https://kai0721300033.cognitiveservices.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"
INFO demo: ========== KAHUA REQUEST (list_projects) ==========
INFO demo: URL: https://devdailyservice.kahua.com/v2/domains/AWrightCo/projects/0/query?returnDefaultAttributes=true
INFO demo: Headers:
{
  "Content-Type": "application/json",
  "Authorization": "YXdyaWdodE...redacted"
}
INFO demo: Payload:
{
  "PropertyName": "Query",
  "EntityDef": "kahua_Project.Project"
}
INFO demo: ===================================================
INFO httpx: HTTP Request: POST https://devdailyservice.kahua.com/v2/domains/AWrightCo/projects/0/query?returnDefaultAttributes=true "HTTP/1.1 401 Unauthorized"
INFO demo: ========== KAHUA RESPONSE (list_projects) ===

I encountered an authorization error while trying to list projects in Kahua. Please verify your permissions or provide updated access credentials.

--- DEMO B: Create project ---


INFO httpx: HTTP Request: POST https://kai0721300033.cognitiveservices.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"
INFO httpx: HTTP Request: POST https://kai0721300033.cognitiveservices.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"
INFO demo: ========== KAHUA REQUEST (create_project) =========
INFO demo: URL: https://devdailyservice.kahua.com/v2/domains/AWrightCo/projects/0/apps/kahua_Project/activities/run
INFO demo: Headers:
{
  "Content-Type": "application/json",
  "Authorization": "YXdyaWdodE...redacted"
}
INFO demo: Payload:
{
  "activity": {
    "PropertyName": "Activity",
    "Name": "ContractItem",
    "Flow": [
      {
        "PropertyName": "Iterate",
        "Set": "ContractItem",
        "New": {},
        "Existing": {}
      }
    ]
  },
  "sets": [
    {
      "name": "ContractItem",
      "entities": [
        {
          "id": 0,
          "hubPath": "kahu

id=None name='East Tower Renovation' description='Demo via tool' auto_send=True
